<a href="https://colab.research.google.com/github/airballoh87/bexa/blob/main/Run_painted_tree_bexa_boutique_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [2]:
##data and stats
import pandas as pd
import seaborn as sns

##visualis
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker  # Import the ticker module

##data sources
import gspread
from google.colab import drive

##web scraping
import requests
from bs4 import BeautifulSoup

##machine learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

##NLP
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk

##saving models
import joblib

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Get data

## Real Data

### Web Scrape from Painted Tree = Cincy only

In [3]:
# Your login credentials
USERNAME = '105357'
PASSWORD = 'WinstonWillow1!'

# Headers for HTTP requests
http_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}

# Initialize an empty DataFrame to accumulate the data
accumulated_data = pd.DataFrame()

# Start a session to keep the cookies
with requests.Session() as session:
    # Prepare login data
    login_data = {
        'consignorId': USERNAME,
        'password': PASSWORD
    }

    # Send a POST request to the login endpoint
    login_url = 'https://cincinnati.consignoraccess.com/auth'  # Adjust this URL if necessary
    login_response = session.post(login_url, headers=http_headers, data=login_data)

    # Verify login by checking the response and cookies
    print("Login response status code:", login_response.status_code)
    print("Cookies after login:", session.cookies.get_dict())

    # Proceed to the data URL
    #data_url = 'https://cincinnati.consignoraccess.com/activity'
    data_url = 'https://cincinnati.consignoraccess.com/activity?fromDate=01/01/2023&toDate=06/30/2024&showPaidItems=true'
    response = session.get(data_url, headers=http_headers)

    # Print the response status code and content for verification
    print("Data URL response status code:", response.status_code)
    print("Data URL response content snippet:", response.content[:500])  # Print first 500 characters of the response content

    soup = BeautifulSoup(response.content, 'html.parser')
    #print("Soup object:", soup.prettify()[:500])  # Print first 500 characters of the parsed HTML

    table = soup.find('table', {'id': 'make-responsive'})

    if table:
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = []
        for row in table.find('tbody').find_all('tr'):
            columns = row.find_all('td')
            if columns:
                rows.append([col.text.strip() for col in columns])
        accumulated_data = pd.DataFrame(rows, columns=headers)

      # Add new columns 'Category', 'Type', and 'Theme' with blank values
        accumulated_data['Category'] = ''
        accumulated_data['Type'] = ''
        accumulated_data['Theme'] = ''
        accumulated_data['Location'] = 'Cincinnati'

        print("Accumulated data:", accumulated_data)
    else:
        print("Table not found in the page.")

    # Optionally save to CSV
    # accumulated_data.to_csv("sales_activity_data.csv", index=False)


Login response status code: 200
Cookies after login: {'key': 'JLz6qc5aEMXS1xRdzKMn'}
Data URL response status code: 200
Data URL response content snippet: b'\n\n<!DOCTYPE html>\n<html lang="en">\n\t<head>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=EDGE"/>\n\t\t<title>Consignor Access - Activity</title>\n\n\t\t<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>\n\t\t<meta content="Consignor Access" name="description"/>\n\t\t<meta content="SimpleConsign" name="author"/>\n\n        \n\t\t\n\t\t\n\n\t\t<link rel="shortcut icon" href="/assets/favicon-9f8530d404b76a186ff295fe37f2b620.ico" type="image/x-icon">\n\n  \t\t<link re'
Accumulated data:            Date                                        Description     SKU  \
0     6/23/2024                                      Straw Toppers  55WQP5   
1     6/23/2024                 Silver Tiger Head 3D stud earrings  JJBBT9   
2     6/23/2024                                   HJ

### Real data - Kentucky - from google sheet


In [4]:
gc  = gspread.authorize(creds)

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gzZ4ffnM5Uc4cCHmAigMQ4IS3xxVokBc1wM5tQjPQcM/edit?gid=85248025#gid=85248025') ##Kentucky
sheets = gsheets.worksheet('Painted Tree Category From Website - Kentucky').get_all_values()
kentucky_sales_data = pd.DataFrame(sheets[1:], columns=sheets[0])
kentucky_sales_data

,Date,Description,SKU,Invoice,Retail Price,Sold Price,Store Amount,Consignor Amount,Category,Type,Theme,Location
0,5/31/24,HEADBAND LUXURY STYLE - SALE CLEARANCE AS-IN -...,V67JCR,50020,$10.00,$10.00,-$1.00,$9.00,,,,Kentucky
1,5/31/24,kids headband - Ric Rac Rainbow Knotted Headband,E6SD2W,50020,$12.00,$12.00,-$1.20,$10.80,,,,Kentucky
2,5/31/24,kids - pink tulle headband with bow,VDKNDW,49917,$14.00,$14.00,-$1.40,$12.60,,,,Kentucky
3,5/31/24,kids - pink tulle headband with bow,VDKNDW,50026,$14.00,$14.00,-$1.40,$12.60,,,,Kentucky
4,5/31/24,bag - Swiftie Canvas bag,K1XNAV,49917,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky
...,...,...,...,...,...,...,...,...,...,...,...,...
2160,6/22/24,headband - purple gemstone embellished knotted,R9UD13,54938,$25.00,$25.00,-$2.50,$22.50,,,,Kentucky
2161,6/22/24,bride rhinestone sunglasses,GYX2AP,55194,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
2162,6/22/24,headband - American USA Flag Colored Knot Sequ...,R9LC27,55047,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
2163,6/22/24,earrings - orange cut out side tiger face earr...,L5H23K,55199,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky


###merge data together

In [5]:
# Concatenate accumulated_data with historical_sales_data as union
union_data = pd.concat([accumulated_data, kentucky_sales_data], axis=0, ignore_index=True)
union_data

,Date,Description,SKU,Invoice,Retail Price,Sold Price,Store Amount,Consignor Amount,Category,Type,Theme,Location
0,6/23/2024,Straw Toppers,55WQP5,85005,$5.00,$5.00,-$0.50,$4.50,,,,Cincinnati
1,6/23/2024,Silver Tiger Head 3D stud earrings,JJBBT9,84995,$15.00,$15.00,-$1.50,$13.50,,,,Cincinnati
2,6/23/2024,HJNPHP bag strap,2NAZ5W,85041,$60.00,$60.00,-$6.00,$54.00,,,,Cincinnati
3,6/23/2024,Pink/Yellow/Champagne Sequin & Beaded Large To...,NV7VAG,84936,$75.00,$75.00,-$7.50,$67.50,,,,Cincinnati
4,6/22/2024,PIn - Dog Lover Pin,2TESGR,84815,$6.50,$6.50,-$0.65,$5.85,,,,Cincinnati
...,...,...,...,...,...,...,...,...,...,...,...,...
4819,6/22/24,headband - purple gemstone embellished knotted,R9UD13,54938,$25.00,$25.00,-$2.50,$22.50,,,,Kentucky
4820,6/22/24,bride rhinestone sunglasses,GYX2AP,55194,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
4821,6/22/24,headband - American USA Flag Colored Knot Sequ...,R9LC27,55047,$20.00,$20.00,-$2.00,$18.00,,,,Kentucky
4822,6/22/24,earrings - orange cut out side tiger face earr...,L5H23K,55199,$15.00,$15.00,-$1.50,$13.50,,,,Kentucky


#Clean Data

## rows and columns - general

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import joblib


# Load the dataset
new_sales_data = union_data

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

new_sales_data['Retail Price'] = new_sales_data['Retail Price'].apply(clean_price)
new_sales_data['Sold Price'] = new_sales_data['Sold Price'].apply(clean_price)
new_sales_data['Store Amount'] = new_sales_data['Store Amount'].apply(clean_price)
new_sales_data['Consignor Amount'] = new_sales_data['Consignor Amount'].apply(clean_price)

# Drop rows with missing values in the target column 'Category'
new_sales_data = new_sales_data.dropna(subset=['Category'])

#Eliminates historical rows where the was a return
new_sales_data = new_sales_data[(new_sales_data['Sold Price'] >= 0) #&
                                #(new_sales_data['Store Amount'] >= 0) &
                                #(new_sales_data['Consignor Amount'] >= 0)
                                ]


# Clean up the 'Category' column
new_sales_data['Category'] = new_sales_data['Category'].str.upper()

# Make specified updates to the 'Category' column
category_updates = {
    'MISC': 'MISCELLANEOUS',
    'SHELL': 'DECOR',
    'CLOTHES': 'CLOTHING',
    'HAT': 'HATS',
    'APPAREL': 'CLOTHING',
    'DECOR - EASTER': 'DECOR',
    'DECOR - VALENTINES': 'DECOR',
    'DOGS': 'PET',
    'HAIR ACCESSORIES': 'ACCESSORIES: HAIR',
    'HOLIDAY - EASTER': 'DECOR',
    'HOLIDAY DECOR - EASTER': 'DECOR',
    'HOME KITCHEN': 'HOME',
    'KIDS': 'KIDS ACCESSORIES',
    'KIDS APPAREL': 'KIDS CLOTHING',
    'KIDS HAIR': 'KIDS ACCESSORIES',
    'KIDS HAT': 'KIDS ACCESSORIES',
    'KIDS HATS': 'KIDS ACCESSORIES',
    'KIDS JEWELRY': 'KIDS ACCESSORIES'
}


type_updates = {

    'BUNNY': 'BUNNY DECOR',
    'COAT': 'JACKET',
    'COVER UP': 'COVERUP',
    'COWBOY HAT': 'COWBOY',
    'COWBOY HAT DECOR': 'COWBOY',
    'DOG COLLAR LARGE': 'DOG COLLAR',
    'DOG COLLAR MEDIUM': 'DOG COLLAR',
    'DOG COLLAR SMALL' : 'DOG COLLAR',
    'EAR MUFFS': 'EARMUFFS',
    'HAIR CLIP': 'HAIR CLIPS',
    'KEY CHAIN': 'KEYCHAIN',
    'ORNAMENT': 'ORNAMENTS',
    'WINTER HAT': 'WINTER',
    'WINE TUMBLER': 'TUMBLER',
    'TOTE': 'TOTE BAG',
    'SWIM': 'SWIMWEAR',
    'RING': 'RINGS',
    'BEADED NECKLACE': 'NECKLACE',
    'BOTTLE OPENER':'MISCELLANEOUS' ,
    'SPPONS': 'SPOONS',
    'SPOON HOLDER':'SPOONS'

}

theme_updates = {
    'ANIMAL PRINT': 'ANIMALS',
    'ART DECO': 'ART',
    'EMBELLISHED DENIM': 'EMBELLISHED',
    'FLOWER': 'FLORAL',
    'FLOWERS': 'FLORAL',
    'HORSE': 'HORSES',
    'MOTHERS DAY': 'MOM',
    'MUSHROOM': 'MUSHROOMS',
    'PLANTS': 'PLANT',
    'RHINESTONE': 'RHINESTONES',
    'SELF ESTEEM': 'SELF CARE',
    'ST. PATRICKS':    'ST PATRICKS DAY',
    'ST.PATRICKS DAY': 'ST PATRICKS DAY',
    'SWIFTIE': 'TAYLOR SWIFT',
    'TIGER':'TIGER/BENGALS',
    'BENGALS': 'TIGER/BENGALS',
    'VALENTINE''S DAY':'VALENTINES',
    'VALENTINE': 'VALENTINES',
    'WINTER COAT':'WINTER',
    'WESTER':'WESTERN'
}

new_sales_data['Category'] = new_sales_data['Category'].replace(category_updates)
new_sales_data['Type'] = new_sales_data['Type'].replace(type_updates)
new_sales_data['Theme'] = new_sales_data['Theme'].replace(theme_updates)

new_sales_data

,Date,Description,SKU,Invoice,Retail Price,Sold Price,Store Amount,Consignor Amount,Category,Type,Theme,Location
0,6/23/2024,Straw Toppers,55WQP5,85005,5.0,5.0,-0.50,4.50,,,,Cincinnati
1,6/23/2024,Silver Tiger Head 3D stud earrings,JJBBT9,84995,15.0,15.0,-1.50,13.50,,,,Cincinnati
2,6/23/2024,HJNPHP bag strap,2NAZ5W,85041,60.0,60.0,-6.00,54.00,,,,Cincinnati
3,6/23/2024,Pink/Yellow/Champagne Sequin & Beaded Large To...,NV7VAG,84936,75.0,75.0,-7.50,67.50,,,,Cincinnati
4,6/22/2024,PIn - Dog Lover Pin,2TESGR,84815,6.5,6.5,-0.65,5.85,,,,Cincinnati
...,...,...,...,...,...,...,...,...,...,...,...,...
4819,6/22/24,headband - purple gemstone embellished knotted,R9UD13,54938,25.0,25.0,-2.50,22.50,,,,Kentucky
4820,6/22/24,bride rhinestone sunglasses,GYX2AP,55194,20.0,20.0,-2.00,18.00,,,,Kentucky
4821,6/22/24,headband - American USA Flag Colored Knot Sequ...,R9LC27,55047,20.0,20.0,-2.00,18.00,,,,Kentucky
4822,6/22/24,earrings - orange cut out side tiger face earr...,L5H23K,55199,15.0,15.0,-1.50,13.50,,,,Kentucky


#Run the model


In [8]:
#import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
import joblib
from google.colab import drive


union_data = new_sales_data

# Ensure NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\W+', ' ', text)  # Remove punctuation
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(words)


# Filter out rows where 'Retail Price' has a value of 'Retail Price'
model_output = union_data[union_data['Retail Price'] != 'Retail Price']

model_output = model_output.drop(columns=['Type', 'Theme', 'Category'])

# Clean and convert price columns to numerical values
def clean_price(price):
    if isinstance(price, str):
        price = price.replace('(', '').replace(')', '').replace('$', '').replace(',', '')
        try:
            return float(price)
        except ValueError:
            return None
    return price

model_output['Retail Price'] = model_output['Retail Price'].apply(clean_price)
model_output['Sold Price'] = model_output['Sold Price'].apply(clean_price)
model_output['Store Amount'] = model_output['Store Amount'].apply(clean_price)
model_output['Consignor Amount'] = model_output['Consignor Amount'].apply(clean_price)

# Apply preprocessing to the 'Description' column
model_output['Description'] = model_output['Description'].apply(preprocess_text)


######category
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_category.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_category.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Category'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Category_prediction_probability'] = max_probabilities

######Type
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_type.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_type.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Type'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Type_prediction_probability'] = max_probabilities


######Theme
# Load the saved TF-IDF vectorizer
tfidf_vectorizer = joblib.load('/content/drive/My Drive/Bexa/tfidf_vectorizer_item_theme.pkl')
# Load the pipeline (if needed)
pipeline = joblib.load('/content/drive/My Drive/Bexa/best_model_theme.pkl')
# Transform the 'Description' column using the loaded TF-IDF vectorizer
X_accumulated_tfidf = tfidf_vectorizer.transform(model_output['Description'])
# Make predictions on the new data
model_output['Predicted_Theme'] = pipeline.predict(model_output['Description'])
# Calculate prediction probabilities
pred_probabilities = pipeline.predict_proba(model_output['Description'])
# Get the maximum probability for each prediction
max_probabilities = pred_probabilities.max(axis=1)
# Add the probabilities to the DataFrame
model_output['Theme_prediction_probability'] = max_probabilities



#####final cleanup

# Remove all '#' from the 'Consignor Amount' column, replace empty strings with '0', and convert to float
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('[\$,#]', '', regex=True)
model_output['Consignor Amount'] = model_output['Consignor Amount'].replace('', '0').astype(float)


# Function to ensure four-digit year
def ensure_four_digit_year(date_str):
    # Match two-digit year and convert to four-digit year
    date_str = re.sub(r'(\d{1,2}/\d{1,2}/)(\d{2})$', r'\g<1>20\2', date_str)
    return date_str

# Apply the function to the 'Date' column
model_output['Date'] = model_output['Date'].apply(ensure_four_digit_year)

# Convert the adjusted date strings to datetime objects
model_output['Date'] = pd.to_datetime(model_output['Date'], format='%m/%d/%Y', errors='coerce')

# Display the first 20 rows of the combined data
print(model_output.head(20))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         Date                                        Description     SKU  \
0  2024-06-23                                       straw topper  55WQP5   
1  2024-06-23                     silver tiger head stud earring  JJBBT9   
2  2024-06-23                                   hjnphp bag strap  2NAZ5W   
3  2024-06-23  pink yellow champagne sequin beaded large tote...  NV7VAG   
4  2024-06-22                                  pin dog lover pin  2TESGR   
5  2024-06-22           bracelet red white blue stretch bracelet  THYGCK   
6  2024-06-22                         pink tiger acrylic earring  1HGLFS   
7  2024-06-22                                 earring small stud  1P8ABR   
8  2024-06-22                          huhhle hoop golod earring  R67TY3   
9  2024-06-22  kid hat kid mini era embroidered baseball cap ...  SXS8C9   
10 2024-06-22           earring mar

#write to file on google sheets

In [9]:

# Assuming 'y_train' is a Series, convert it to a DataFrame for easier handling
model_output = model_output.astype(str)

# Convert the DataFrame to a list of lists
data = [model_output.columns.values.tolist()] + model_output.values.tolist()

# Create a new Google Sheet or open an existing one
spreadsheet_name = 'painted_tree_ml_training'
worksheet_name = 'Painted Tree - New model output June final'

# Try to open the existing spreadsheet
try:
    spreadsheet = gc.open(spreadsheet_name)
except gspread.SpreadsheetNotFound:
    spreadsheet = gc.create(spreadsheet_name)

# Check if the worksheet already exists and delete it if it does
try:
    worksheet = spreadsheet.worksheet(worksheet_name)
    spreadsheet.del_worksheet(worksheet)
except gspread.WorksheetNotFound:
    pass

# Add a new worksheet
worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=len(model_output)+1, cols=len(model_output.columns))
#worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=len(y_train_df)+1, cols=len(y_train_df.columns))

# Write data to the worksheet
worksheet.update('A1', data)


<ipython-input-9-4841021563e6>:29: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data)


{'spreadsheetId': '1gzZ4ffnM5Uc4cCHmAigMQ4IS3xxVokBc1wM5tQjPQcM',
 'updatedRange': "'Painted Tree - New model output June final'!A1:O4755",
 'updatedRows': 4755,
 'updatedColumns': 15,
 'updatedCells': 71325}

#data viz